In [78]:
dataPath = 'Z://HarveyLab/Alan/Data/20170927/p1'

In [79]:
import os
os.chdir(dataPath)
import sys
kwikToolsPath = 'C://Users/Alan/Documents/Github/kwik-tools'
sys.path.append(kwikToolsPath)
import read_rhd as rhd
import numpy as np
import glob
import matplotlib.pyplot as plt
from shutil import copy2
%matplotlib inline

In [80]:
datFiles = glob.glob('*.dat')
for x in datFiles:
    print(x)
datFiles.sort(key=os.path.getmtime)
print('\nSorted Files:')
for x in datFiles:
    print(x)

gridIndent_170927_153916.dat
gridIndent_170927_152759.dat
gridIndent_170927_155209.dat
gridIndent_170927_153503.dat
gridIndent_170927_152052.dat
gridIndent_170927_154740.dat
gridIndent_170927_155725.dat
gridIndent_170927_154946.dat
gridIndent_170927_155034.dat
gridIndent_170927_154915.dat
gridIndent_170927_153344.dat
gridIndent_170927_152139.dat
gridIndent_170927_153140.dat
gridIndent_170927_153021.dat
gridIndent_170927_154432.dat
gridIndent_170927_155018.dat
gridIndent_170927_155121.dat
gridIndent_170927_155224.dat
gridIndent_170927_153005.dat
gridIndent_170927_152639.dat
gridIndent_170927_152727.dat
gridIndent_170927_152504.dat
gridIndent_170927_153607.dat
gridIndent_170927_154122.dat
gridIndent_170927_154051.dat
gridIndent_170927_152830.dat
gridIndent_170927_152417.dat
gridIndent_170927_155859.dat
gridIndent_170927_154653.dat
gridIndent_170927_155503.dat
gridIndent_170927_153037.dat
gridIndent_170927_154257.dat
gridIndent_170927_155153.dat
gridIndent_170927_153812.dat
gridIndent_170

In [3]:
files = glob.glob('*.rhd')
for x in files:
    print(x)
files.sort(key=os.path.getmtime)  ## this sorting may be OS sensitive
print('\nSorted Files:')
for x in files:
    print(x)

indenter_170911_105716.rhd
optoTest2_170911_110103.rhd
indenter_170911_105548.rhd
indenter_170911_105906.rhd
indenter2_170911_110751.rhd
optoTest2_170911_110015.rhd
optoTest3_170911_111201.rhd
indenter2_170911_110919.rhd
indenter2_170911_110707.rhd
indenter2_170911_110601.rhd
indenter_170911_105610.rhd
optoTest2_170911_110003.rhd
optoTest3_170911_111125.rhd
optoTest1_170911_104218.rhd
optoTest2_170911_110151.rhd
optoTest1_170911_104242.rhd
optoTest3_170911_111113.rhd
optoTest2_170911_110027.rhd
indenter_170911_105358.rhd
indenter_170911_105420.rhd
optoTest2_170911_110039.rhd
indenter_170911_105738.rhd
indenter_170911_105314.rhd
indenter_170911_105632.rhd
indenter_170911_105822.rhd
indenter2_170911_111025.rhd
indenter2_170911_110433.rhd
indenter_170911_105654.rhd
indenter2_170911_110455.rhd
optoTest1_170911_104154.rhd
indenter2_170911_110857.rhd
optoTest3_170911_111237.rhd
indenter2_170911_110623.rhd
optoTest2_170911_110127.rhd
indenter_170911_105230.rhd
optoTest3_170911_111137.rhd
inde

In [24]:
basename = os.path.basename(datapath) # taking foldername as basename

#batch write .rhd to .dat
for file in files:
    d = rhd.read_rhd(file)
    recordings = np.transpose(d['amplifier_data'])
    stimulation = ()
    #save amplifier data as .dat
    recordings.tofile(os.path.splitext(file)[0] +'.dat')  
    #save digital input streams
    for digitalChannel in range(d['board_dig_in_data'].shape[0]):
        d['board_dig_in_data'][digitalChannel].tofile(os.path.splitext(file)[0] + 'chan' + str(digitalChannel) + '.di')
    #save analog input streams
    for analogChannel in range(d['aux_input_data'].shape[0]):
        d['aux_input_data'][analogChannel].tofile(os.path.splitext(file)[0] + 'chan' + str(analogChannel) + '.ai')

#create .prm file and copy .prb file to data directory    
datFiles = glob.glob(basename + '*.dat')
datFiles.sort(key=os.path.getmtime)

prb_file = 'A1x32-Poly2-5mm-50s-177-A32.prb' #prb_file = 'Buzsaki32.prb'
copy2(kwikToolsPath+'/'+prb_file,dataPath)


with open(basename + '.prm', "w") as text_file:
        text_file.write('experiment_name = \'{0}\' \n'.format(basename))
        text_file.write('prb_file = \'{0}\'\n'.format(prb_file)) 
        text_file.write('traces = dict( \n')
        text_file.write('\traw_data_files={0},\n'.format(datFiles))
        text_file.write('\tvoltage_gain={0}.0,\n'.format(192))  #from Intan RHD2000 documentation
        text_file.write('\tsample_rate={0},\n'.format(d['frequency_parameters']['amplifier_sample_rate']))
        text_file.write('\tn_channels={0},\n'.format(len(d['amplifier_channels'])))
        text_file.write('\tdtype=\'uint16\'\n')
        text_file.write('\t)\n\n')
        
        text_file.write('spikedetekt = dict(\n')
        text_file.write('\tfilter_low=500.,\n') # low pass filter? (as documented in phy)
        text_file.write('\tfilter_high_factor=0.95 * .5,\n') # high pass filter (as documented in phy)
        text_file.write('\tfilter_butter_order=3,\n') # order of Butterworth Filter
        text_file.write('\n')
        text_file.write('\tfilter_lfp_low=20,\n') # LFP filter low-pass frequency
        text_file.write('\tfilter_lfp_high=0,\n') #LFP filter high-pass frequency
        text_file.write('\n')
        text_file.write('\tchunk_size_seconds=1,\n')
        text_file.write('\tchunk_overlap_seconds=.015,\n')
        text_file.write('\n')
        text_file.write('\tn_excerpts=50,\n')
        text_file.write('\texcerpts_size_seconds=1,\n')
        text_file.write('\tthreshold_strong_std_factor=5.5,\n')
        text_file.write('\tthreshold_weak_std_factor=3.5,\n')
        text_file.write("\tdetect_spikes='negative',\n")
        text_file.write('\n')
        text_file.write('\tconnected_component_join_size=1,\n')
        text_file.write('\n')
        text_file.write('\textract_s_before=16,\n')
        text_file.write('\textract_s_after=16,\n')
        text_file.write('\n')
        text_file.write('\tn_features_per_channel=3,\n') #number of features per channel
        text_file.write('\tpca_n_waveforms_max=10000,\n')
        text_file.write(')\n')
        text_file.write('\n')
        text_file.write('klustakwik2 = dict(\n')
        text_file.write('\tnum_starting_clusters=100,\n')
        text_file.write(')\n')
    




Reading Intan Technologies RHD2000 Data File, Version 1.5

Found 32 amplifier channels.
Found 3 auxiliary input channels.
Found 1 supply voltage channel.
Found 0 board ADC channels.
Found 2 board digital input channels.
Found 0 board digital output channels.
Found 0 temperature sensors channels.

File contains 59.976 seconds of data.  Amplifiers were sampled at 20.00 kS/s.

Allocating memory for data...
Reading data from file...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Parsing data...
No missing timestamps in data.
Applying notch filter...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Done!  Elapsed time: 6.1 seconds

Reading Intan Technologies RHD2000 Data File, Version 1.5

Found 32 amplifier channels.
Found 3 auxiliary input channels.
Found 1 supply voltage channel.
Found 0 board ADC channels.
Found 2 board digital input channels.
Found 0 board digital ou

# concatenate all .dat files for mda conversion for sorting with mountainsort

In [82]:
## Concatenate dat files prior to mda conversion

allData = np.array([])
numFiles = len(datFiles)

for i, file in enumerate(datFiles):
        print('on file',i,'of',numFiles)
        allData = np.concatenate((allData, np.fromfile(file)))
        
if not os.path.exists(dataPath+'/alldata'):
    os.mkdir(dataPath+'/alldata')

allData.tofile(dataPath+'/alldata/allData.dat')

# use this as second dimension when converting to mda file using mdaconvert
print('use this as second dimension when converting to mda file using mdaconvert',len(allData)/8)

on file 0 of 147
on file 1 of 147
on file 2 of 147
on file 3 of 147
on file 4 of 147
on file 5 of 147
on file 6 of 147
on file 7 of 147
on file 8 of 147
on file 9 of 147
on file 10 of 147
on file 11 of 147
on file 12 of 147
on file 13 of 147
on file 14 of 147
on file 15 of 147
on file 16 of 147
on file 17 of 147
on file 18 of 147
on file 19 of 147
on file 20 of 147
on file 21 of 147
on file 22 of 147
on file 23 of 147
on file 24 of 147
on file 25 of 147
on file 26 of 147
on file 27 of 147
on file 28 of 147
on file 29 of 147
on file 30 of 147
on file 31 of 147
on file 32 of 147
on file 33 of 147
on file 34 of 147
on file 35 of 147
on file 36 of 147
on file 37 of 147
on file 38 of 147
on file 39 of 147
on file 40 of 147
on file 41 of 147
on file 42 of 147
on file 43 of 147
on file 44 of 147
on file 45 of 147
on file 46 of 147
on file 47 of 147
on file 48 of 147
on file 49 of 147
on file 50 of 147
on file 51 of 147
on file 52 of 147
on file 53 of 147
on file 54 of 147
on file 55 of 147
on